In [1]:
!pip install kaggle
import kaggle
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

     ---------------------------------------- 0.0/82.1 kB ? eta -:--:--
     ------------------ ------------------- 41.0/82.1 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 82.1/82.1 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105130 sha256=3572024baa494b0a8ff13fa2c23a270ff720a0c651a6b3376987f223bc43a114
  Stored in directory: c:\users\tup30579\appdata\local\pip\cache\wheels\ec\d9\04\93be57edf3835182a8e76d8ff87f7e77a0113f473ba3721ee0
Successfully built kaggle
Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:00<00:00, 572kB/s]
100%|##########| 200k/200k [00:00<00:00, 570kB/s]


In [2]:
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [6]:
import pandas as pd
df = pd.read_csv('orders.csv',na_values=['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [11]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','_')

In [16]:
df['discount']=df['list_price']*df['discount_percent']*0.01
df['sale_price']=df['list_price']-df['discount']
df['profit']=df['sale_price']-df['cost_price']
df['margin']=100*df['profit']/df['cost_price']

In [18]:
df.dtypes

order_id              int64
order_date           object
ship_mode            object
segment              object
country              object
city                 object
state                object
postal_code           int64
region               object
category             object
sub_category         object
product_id           object
cost_price            int64
list_price            int64
quantity              int64
discount_percent      int64
discount            float64
sale_price          float64
profit              float64
margin              float64
dtype: object

In [19]:
df['order_date']=pd.to_datetime(df['order_date'],format="%Y-%m-%d")

In [22]:
!pip install pandas-gbq google-auth

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/193.7 kB ? eta -:--:--
   --------------------------------------- 193.7/193.7 kB 12.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/139.0 kB ? eta -:--:--
   ---------------------------------------- 139.0/139.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/238.5 kB ? eta -:--:--
   --------------------------------------- 238.5/238.5 kB 15.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/81.2 kB ? eta -:--:--
   ---------------------------------------- 81.2/81.2 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/229.2 kB ? eta -:--:--
   ---------------------------------------- 229.2/229.2 kB ? eta 0:00:00
   ---------------------------------------- 0.0/48.8 kB ? eta -:--:--
   ---------------------------------------

In [26]:
from google.oauth2 import service_account
from pandas_gbq import to_gbq

# Path to your service account key file
key_path = r"C:\Users\tup30579\BigQuery\totemic-web-426306-i7-6e151b9ce7f7.json"

# Authenticate using the service account key
credentials = service_account.Credentials.from_service_account_file(key_path)

# Your Google Cloud Project ID
project_id = 'totemic-web-426306-i7'

In [28]:
table_id = 'my_dataset.orders_cleaned'
to_gbq(df, table_id, project_id=project_id, if_exists='replace', credentials=credentials)

100%|██████████| 1/1 [00:00<?, ?it/s]


In [29]:
df.head(20)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit,margin
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8,6.166667
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1,18.016667
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5,-5.000000
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8,20.615385
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0,-5.000000
5,6,2022-03-13,NaN,Consumer,United States,Los Angeles,California,90032,West,Furniture,Furnishings,FUR-FU-10001487,50,50,7,3,1.5,48.5,-1.5,-3.000000
6,7,2022-12-28,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Art,OFF-AR-10002833,10,10,4,3,0.3,9.7,-0.3,-3.000000
7,8,2022-01-25,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Technology,Phones,TEC-PH-10002275,860,910,6,5,45.5,864.5,4.5,0.523256
8,9,2023-03-23,NaN,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Binders,OFF-BI-10003910,20,20,3,2,0.4,19.6,-0.4,-2.000000
9,10,2023-05-16,Standard Class,Consumer,United States,Los Angeles,California,90032,West,Office Supplies,Appliances,OFF-AP-10002892,90,110,5,3,3.3,106.7,16.7,18.555556
